In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

HEADERS ={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

# Specify the URL to scrape
url = 'https://www.allrecipes.com/recipes/1290/everyday-cooking/more-meal-ideas/30-minute-meals/vegetarian/'


response = requests.get(url, headers=HEADERS)


soup = BeautifulSoup(response.content, 'html.parser')

recipee_name_list = list()
recipee_links = list()
ingredients_list = list()
directions_list = list()
facts_list = list()



links = soup.find_all("a", {"class": "comp card--image-top mntl-card-list-items mntl-document-card mntl-card card card--no-image"})
recipees = soup.find_all("span", {"class": "card__title"})
for i in range(3):
  recipee_name_list.append(recipees[i].get_text())

for link in links:
  recipee_links.append(link.get("href"))


#This loop is finding ingredients
for link in links:
  
  recipee_response = requests.get(link.get("href"), headers=HEADERS)
  new_soup = BeautifulSoup(recipee_response.content, 'html.parser')

  string = ""
  for quantity in new_soup.find_all("li", {"class": "mntl-structured-ingredients__list-item"}):
    for span in quantity.find_all("span"):
      string = string + span.text + ", "

  ingredients_list.append(string)


#This loop is finding directions
for link in links:

  recipee_response = requests.get(link.get("href"), headers=HEADERS)
  new_soup = BeautifulSoup(recipee_response.content, 'html.parser')

  string = ""
  current_step = 1
  for step in new_soup.find_all("li", {"class": "comp mntl-sc-block-group--LI mntl-sc-block mntl-sc-block-startgroup"}):
    for direction in step.find_all("p"):
      string = string + "Step " +str(current_step) + " : "
      string = string + direction.text + ", "
      current_step += 1

  directions_list.append(string)


# This loop is finding nutrition facts
for link in links:

  recipee_response = requests.get(link.get("href"), headers=HEADERS)
  new_soup = BeautifulSoup(recipee_response.content, 'html.parser')

  string = ""
  count = 0
  for step in new_soup.find_all("td", {"class": "mntl-nutrition-facts-summary__table-cell type--dog-bold"}):
    #print(step.text)
    string = string + step.text + " "
    if count == 0:
      string = string + "Calories"
      count += 1
    elif count == 1:
      string = string + "fat"
      count += 1
    elif count == 2:
      string = string + "carbs"
      count += 1
    else:
      string = string + "protein"
      count = 0
    
    string = string + ", "
  
  facts_list.append(string)

In [ ]:
import pandas as pd

data_frame = pd.DataFrame({
    "Recipee Name" : recipee_name_list,
    "Recipee Links" : recipee_links,
    "Ingredients" : ingredients_list,
    "Directions" : directions_list,
    "Nutritional facts" : facts_list
})

data_frame

,Recipee Name,Recipee Links,Ingredients,Directions,Nutritional facts
0,Spinach-Artichoke Garlic Naan Pizza,https://www.allrecipes.com/recipe/8462067/spin...,"1, (9 inch), garlic naan bread, 2 ½, tablespoo...",Step 1 : \nPreheat the oven to 400 degrees F (...,"301 Calories, 19g fat, 23g carbs, 12g prote..."
1,Gochujang Fried Rice,https://www.allrecipes.com/recipe/8527056/goch...,"2, tablespoons, vegetable oil, 2, teaspoons, m...",Step 1 : \nHeat a wok or large skillet over me...,"340 Calories, 10g fat, 55g carbs, 6g protein,"
2,Black Beans and Rice,https://www.allrecipes.com/recipe/15559/black-...,"1, teaspoon, olive oil, 1, , onion, chopped, 2...",Step 1 : \nHeat oil in a saucepan over medium-...,"140 Calories, 1g fat, 27g carbs, 6g protein,"


In [ ]:
file_name = 'AllRecipes.xlsx'
  
# saving the excel
data_frame.to_excel(file_name)